In [1]:
# Target 預測會不會購買 未來第幾天的銷售量 利用MTL

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [27]:
address = './/---//---//' # 設定地址
A = pd.read_csv('.//南山//新資料1.csv', encoding = 'big5')
B = pd.read_csv('.//南山//新資料2.csv', encoding = 'big5')
C = pd.read_csv(".//南山//2018-2020資料合併.csv")
# 改文字title
C.columns = ['U','MONTH', 'AGENT_DUMMY','SUGGEST_DATE','SUGGEST_LAST_DATE','BP_DUMMY','NAME','AMOUNT','FORTUNELV','LOYALTYLV','CLUSTER','SUGGEST_DUMMY','AGE','TYPE','CATEGORY','E_ACCEPT','ACCEPT','ACCEPT_DUMMY','CURRENCY','FEE','Timespan']
# 選擇有用的欄位
T = C['Timespan']
C = C.iloc[:,[1,2,11,3,4,12,5,6, 13,14,7,18,19,16, 15, 8, 9, 10, 20]]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
print('最後給予的資料之shape：', A.shape)
print("最後給予的資料之shape：", B.shape)
print("原先且已處理的資料之shape：", C.shape) # 多一個column是為了之後要做成交天數而先宣告的一個column

最後給予的資料之shape： (332998, 18)
最後給予的資料之shape： (318553, 18)
原先且已處理的資料之shape： (1117903, 19)


In [33]:
# 將資料
f = pd.concat([A,B,C], axis = 0).iloc[:,:-1].reset_index(drop = True); print('資料合併後的shape：',f.shape)
# 把資料統一（處理掉資料庫本身的資料誤存）
f = f[f['AGE'] != ' ']
f = f[f['AMOUNT'] != '   ']
f = f[f['FEE'] != ' -   ']

資料合併後的shape： (1769454, 18)


In [34]:
# 建議書建立日的異樣格式處理
for i,j in enumerate(f['SUGGEST_DATE']):
    if '/' not in j:
        print(i,j)
        break

645310 2019-04-11 17:48:43


In [35]:
from datetime import datetime
import datetime as dt
# 資料統一化 與 日期轉換成python可以處理的型態
# 645310 是日期格式異樣的始點
a = f.iloc[:645310 ]
b = f.iloc[645310 :]
a['SUGGEST_DATE'] = a['SUGGEST_DATE'].apply(lambda x: datetime.strptime(x, "%Y/%m/%d"))
a['SUGGEST_LAST_DATE'] = a['SUGGEST_LAST_DATE'].apply(lambda x: datetime.strptime(x, "%Y/%m/%d"))

<ipython-input-35-22972b9d4996>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['SUGGEST_DATE'] = a['SUGGEST_DATE'].apply(lambda x: datetime.strptime(x, "%Y/%m/%d"))
<ipython-input-35-22972b9d4996>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['SUGGEST_LAST_DATE'] = a['SUGGEST_LAST_DATE'].apply(lambda x: datetime.strptime(x, "%Y/%m/%d"))


In [36]:
# 處理之函式
def pd_drop(df, col, threshold):
    index = list(b[col]).index(threshold)
    return pd.concat([df.iloc[:index], df.iloc[index+1:]], axis = 0).reset_index(drop = True)

In [37]:
b = pd_drop(b, 'SUGGEST_LAST_DATE', '43944.836006944446') # 資料錯誤，暫時遮掉不討論（只有這一筆錯誤）
# 日期再處理
b['SUGGEST_DATE'] = b['SUGGEST_DATE'].apply(lambda x:datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
b['SUGGEST_LAST_DATE'] = b['SUGGEST_LAST_DATE'].apply(lambda x:datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
a = a.dropna(); b = b.dropna()

In [38]:
# 將字串轉換為數值資料
a['AGE'] = a['AGE'].apply(lambda x: int(x))
a['AMOUNT'] = a['AMOUNT'].apply(lambda x: int(x.split(',')[0] + x.split(',')[1]) if ',' in x else int(x))
a['FEE'] = a['FEE'].apply(lambda x: int(x.split(',')[0] + x.split(',')[1]) if ',' in x else int(x))

In [39]:
# 處理完資料 → 整合
C = pd.concat([C.iloc[:678847,:], C.iloc[678848:,:]], axis = 0).reset_index(drop = True)

In [40]:
# "確定"轉換日期格式
C['SUGGEST_DATE'] = C['SUGGEST_DATE'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
C['SUGGEST_LAST_DATE'] = C['SUGGEST_LAST_DATE'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [41]:
# Dummy資料處理（生成）
f = pd.concat([a,b,C], axis = 0);
D1 = f.drop_duplicates(subset=["BP_DUMMY", "NAME","FEE","SUGGEST_DATE"], keep='last') 
D1 = D1.sort_values("BP_DUMMY").reset_index(drop=True)
DUM = pd.get_dummies(D1[["NAME","TYPE","CATEGORY"]])
DUM=pd.concat([DUM, D1["SUGGEST_DUMMY"]], axis=1)
DUM=DUM.groupby(['SUGGEST_DUMMY']).sum()
DUM = DUM.rename_axis('SUGGEST_DUMMY').reset_index()
print(DUM.shape)
f = f.reset_index(drop = True)

(401874, 162)


In [ ]:
# 承接原始做法
# 將 [建議書] 合併起來
from tqdm import tqdm
from time import sleep
LIST=DUM["SUGGEST_DUMMY"]
i=0
#AGG=D1[0:0]
AGG=pd.DataFrame({'MONTH':[],'AGENT_DUMMY':[],'SUGGEST_DUMMY':[],'SUGGEST_DATE':[],'SUGGEST_LAST_DATE':[],'AGE':[],'BP_DUMMY':[],'AMOUNT':[],'CURRENCY':[],'FEE':[],'E_ACCEPT':[],'ACCEPT':[],'ACCEPT_DUMMY':[],'CLUSTER':[], 'DURATION':[]})
with tqdm(total=len(LIST)-i) as pbar:
    while(i<len(LIST)):
        temp=f[f["SUGGEST_DUMMY"]==str(LIST[i])].reset_index(drop=True)
        MONTH=temp["MONTH"][0]
        AGENT_DUMMY=temp["AGENT_DUMMY"][0]
        SUGGEST_DUMMY=temp["SUGGEST_DUMMY"][0]
        #SUGGEST_DATE=temp["SUGGEST_DATE"][0]
        SUGGEST_DATE = sorted(list(temp["SUGGEST_DATE"]))[0]
        #SUGGEST_LAST_DATE=temp["SUGGEST_LAST_DATE"][len(temp["SUGGEST_LAST_DATE"])-1]
        SUGGEST_LAST_DATE = sorted(list(temp["SUGGEST_LAST_DATE"]))[len(temp["SUGGEST_LAST_DATE"])-1]
        DURATION = temp['Timespan'].iloc[-1]
        try:
            AGE=temp["AGE"].mean()
        except:
            AGE=0
        BP_DUMMY=temp["BP_DUMMY"][0]
        try:
            AMOUNT=temp["AMOUNT"].sum()
        except:
            AMOUNT=0
        CURRENCY=temp["CURRENCY"][0]
        try:
            FEE=temp["FEE"].sum()
        except:
            FEE=0
        E_ACCEPT=temp["E_ACCEPT"][0]
        ACCEPT=temp["ACCEPT"][0]
        ACCEPT_DUMMY=0
        CLUSTER=temp["CLUSTER"][0]
        FORTUNELV = temp['FORTUNELV'][0]
        LOYALTYLV = temp['LOYALTYLV'][0]
        
        AGG=AGG.append(pd.DataFrame({'MONTH':[MONTH],'AGENT_DUMMY':[AGENT_DUMMY],'SUGGEST_DUMMY':[SUGGEST_DUMMY],'SUGGEST_DATE':[SUGGEST_DATE],'SUGGEST_LAST_DATE':[SUGGEST_LAST_DATE],'AGE':[AGE],'BP_DUMMY':[BP_DUMMY],'AMOUNT':[AMOUNT],'CURRENCY':[CURRENCY],'FEE':[FEE],'E_ACCEPT':[E_ACCEPT],'ACCEPT':[ACCEPT],'ACCEPT_DUMMY':[ACCEPT_DUMMY],'CLUSTER':[CLUSTER], 'FORTUNELV':[FORTUNELV], 'LOYALTYLV':[LOYALTYLV],'DURATION':[DURATION]}))
        i+=1
        pbar.update()
AGG

In [ ]:
LIST=data["BP_DUMMY"].unique()
i=0; #ccc = 0
#AGG=D1[0:0]
agg=pd.DataFrame(D)
with tqdm(total=len(LIST)-i) as pbar:
    while(i<len(LIST)):
        Temp=data[data["BP_DUMMY"]==str(LIST[i])].reset_index(drop=True)
        LIST2 = Temp['SUGGEST_DATE'].unique()
        j = 0
        while (j < len(LIST2)):

In [25]:
# 處理跨越問題，同一BP_DUMMY且同一建議日下 視為同一筆資料
# 每一list的位置與Columns 是事先對應好的，部分欄位還需再
LIST=data["BP_DUMMY"].unique()
i=0; #ccc = 0
#AGG=D1[0:0]
agg=pd.DataFrame(D)
with tqdm(total=len(LIST)-i) as pbar:
    while(i<len(LIST)):
        Temp=data[data["BP_DUMMY"]==str(LIST[i])].reset_index(drop=True)
        LIST2 = Temp['SUGGEST_DATE'].unique()
        j = 0
        while (j < len(LIST2)):
            temp = Temp[Temp['SUGGEST_DATE'] == LIST2[j]].reset_index(drop = True)
            Alist = list()
            Alist.append(temp["MONTH"][0])
            Alist.append(temp["AGENT_DUMMY"][0])
            Alist.append(temp["SUGGEST_DUMMY"][0])
            Alist.append(sorted(list(temp["SUGGEST_DATE"]))[0])
            Alist.append(sorted(list(temp["SUGGEST_LAST_DATE"]))[len(temp["SUGGEST_LAST_DATE"])-1])
            #Alist.append((SUGGEST_LAST_DATE - SUGGEST_DATE).days)
            try:
                Alist.append(temp["AGE"].mean())
            except:
                Alist.append(0)
            Alist.append(temp["BP_DUMMY"][0])
            try:
                Alist.append(temp["AMOUNT"].sum())
            except:
                Alist.append(0)
            Alist.append(temp["CURRENCY"][0])
            try:
                Alist.append(temp["FEE"].sum())
            except:
                Alist.append(0)
            Alist.append(temp["E_ACCEPT"][len(temp['E_ACCEPT'])-1])
            Alist.append(temp["ACCEPT"][len(temp['ACCEPT'])-1])
            Alist.append(0) # 是建議書保單
            Alist.append(temp["CLUSTER"][0])
            Alist.append((SUGGEST_LAST_DATE - SUGGEST_DATE).days)
            Alist.append(temp['FORTUNELV'][0])
            Alist.append(temp['LOYALTYLV'][0])
            T = dict()
            for l in data.columns:
                T[l] = []
            T['DURATION'] = []
            for n,k in enumerate(temp.columns):
                if n < 17:
                    T[k] = [Alist[n]]
                else:
                    T[k] = [temp.iloc[-1,n]]
            
            agg=agg.append(pd.DataFrame(T))
            j+=1 
        #ccc += len(LIST2)
        pbar.update()
        i+=1
        #print(i,j)
    agg

  0%|                                                                           | 191/323792 [00:20<9:28:50,  9.48it/s]


KeyboardInterrupt: 

In [ ]:
# 欄位再清理
agg = agg.drop(columns = ['ACCEPT_DUMMY'])

agg即為處理完的資料為：1. 已經刪除重複的建議書。 2. 處理完業務員跨月修改後，建議書DUMMY會更動的問題。